Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue
import math
import numpy as np


In [2]:
PROBLEM_SIZE = 10
NUM_SETS = 50
SETS = tuple(
    np.array([random() < 0.3 for i in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
COUNT_SET = np.array(SETS).sum(axis=0)
LEN_SET = np.array(SETS).sum(axis=1)
State = namedtuple('State', ['taken', 'not_taken'])

In [3]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))


#heuristic functions
def heuristic(heu = "LenDistance", f=lambda x: math.floor(math.sqrt(x)), N=5):
    
    if(heu== "FastDistance"):
        def distance(state):
            covered = reduce(
                    np.logical_or,
                    [SETS[i] for i in state.taken],
                    np.array([False for _ in range(PROBLEM_SIZE)]),
                )
            remaining_sets=list(LEN_SET[list(state.not_taken)])
            remaining_sets.sort(reverse=True)
            to_reach = PROBLEM_SIZE-np.sum(covered)
            reached = 0
            n = 0
            for val in remaining_sets:
                if reached>=to_reach:
                    break
                reached+=val
                n+=1

            return n
        
        return distance


    if(heu == "SimpleDistance"):
        def distance(state):
            if goal_check(state):
                return 0
            for action in state.not_taken:
                state_copy= State(
                        state.taken ^ {action},
                        state.not_taken ^ {action},
                    )
                if goal_check(state_copy):
                    return 1
            return 2
        
        return distance
    

    if(heu == "LenDistance"):
        def distance(state):
            covered = reduce(
                    np.logical_or,
                    [SETS[i] for i in state.taken],
                    np.array([False for _ in range(PROBLEM_SIZE)]),
                )
            to_reach = PROBLEM_SIZE-np.sum(covered)
            if(to_reach==0):
                return 0
            
            not_taked=[np.logical_and(SETS[i], np.logical_not(covered)) for i in state.not_taken]
            not_taked = list(map(lambda x: np.sum(x), not_taked))
            not_taked.sort(reverse=True)
            
            reached = 0
            n = 0
            for val in not_taked:
                if reached>=to_reach:
                    break
                reached+=val
                n+=1

            return n

        
        return distance
    
    if(heu=="ImbalanceDistance"):
        def distance(state):
            covered = reduce(
                    np.logical_or,
                    [SETS[i] for i in state.taken],
                    np.array([False for _ in range(PROBLEM_SIZE)]),
                )
            
            to_reach = PROBLEM_SIZE-np.sum(covered)
            if to_reach==0:
                return 0
            
            to_take = f(to_reach)
            a = list(enumerate(np.where(covered, NUM_SETS + 1, COUNT_SET).tolist()))
            a.sort(key= lambda x: x[1])
            index = list(map(lambda x: x[0], a))[:to_take]
            not_taked_indexed = [SETS[i][index] for i in state.not_taken]
            not_taked_indexed = list(map(lambda x: np.sum(x), not_taked_indexed))
            not_taked_indexed.sort(reverse=True)
            
            taked = 0
            m = 0
            for val in not_taked_indexed:
                if taked>=to_take:
                    break
                taked+=val
                m+=1
            return m

        
        return distance
    
    if(heu=="RandomSamplingDistance"):

        def distance(state):
            covered = reduce(
                    np.logical_or,
                    [SETS[i] for i in state.taken],
                    np.array([False for _ in range(PROBLEM_SIZE)]),
                )
            
            to_reach = PROBLEM_SIZE-np.sum(covered)
            if to_reach==0:
                return 0
            
            a = list(enumerate(covered.tolist()))
            a = np.array(list(map(lambda x: x[0], a)))[covered==False]
            
            maximum=0
            for _ in range(N):
                to_take = f(to_reach)
                a=np.random.permutation(a)
                index = a[:to_take]
                not_taked_indexed = [SETS[i][index] for i in state.not_taken]
                not_taked_indexed = list(map(lambda x: np.sum(x), not_taked_indexed))
                not_taked_indexed.sort(reverse=True)
                
                taked = 0
                m = 0
                for val in not_taked_indexed:
                    if taked>=to_take:
                        break
                    taked+=val
                    m+=1
                maximum= max(maximum, m)

            return m

        return distance
            
def cost(state):
    return len(state.taken)

In [4]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Probelm not solvable"

In [5]:
frontier = PriorityQueue()
def f(x):
    a=math.ceil(np.random.normal(2*x//3, x//4)) 
    if a<1:
        return 1 
    elif a> x:
        return x 
    else:
        return a

distance = heuristic(heu="RandomSamplingDistance", f=f )

state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0
current_total_cost, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        new_total_cost = cost(new_state)+ distance(new_state)
        frontier.put((max(new_total_cost, current_total_cost), new_state))
    _, current_state = frontier.get()

print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)


Solved in 7 steps (2 tiles)


In [6]:
print(f'An optimal solution is: {current_state.taken}')

An optimal solution is: {33, 29}
